In [1]:
from QHD import *

import matplotlib.pyplot as plt
import numpy as np

In [45]:
def morse_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,dt,mass,alp,D):
    x2 = x2 - 4.0*(alp/mass)*x*(xp - x*p)*0.5*dt
    x2 = exp(-2.0*alp*p*dt/mass)*x2
    x2 = x2 - 4.0*(alp/mass)*x*(xp - x*p)*0.5*dt
    return x2

def morse_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,dt,mass,alp,D):
    xp = xp + 0.5*dt*(2.0*alp*D*(x*(3.0*x2 - 2*x*x) - x2)  - (alp*x/mass)*(p2 - 2.0*p*p) )
    xp = exp(-2.0*alp*p*dt/mass)*xp
    xp = xp + 0.5*dt*(2.0*alp*D*(x*(3.0*x2 - 2*x*x) - x2)  - (alp*x/mass)*(p2 - 2.0*p*p) )
    return xp

def gaus_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,dt,mass,alp,D):
    x2 = x2 - (q*x*(xp - x*p))/mass*0.5*dt #4.0*(alp/mass)*x*(xp - x*p)*0.5*dt
    x2 = exp(-2.0*alp*(dt*p/mass)**2)*x2
    x2 = x2 - (q*x*(xp - x*p))/mass*0.5*dt  #4.0*(alp/mass)*x*(xp - x*p)*0.5*dt
    return x2

def gaus_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,dt,mass,alp,D):
    xp = xp + 0.5*dt*(((x*(p2-2*p*p)))/(-2*mass) - (-v_0*x2*2*alpha*q+v_0**2*4*alpha**2*q2*x2))#0.5*dt*(2.0*alp*D*(x*(3.0*x2 - 2*x*x) - x2)  - (alp*x/mass)*(p2 - 2.0*p*p) )
    xp = exp(-2*alpha*(dt*p/mass)**2)*xp
    xp = xp + 0.5*dt*(((x*(p2-2*p*p)))/(-2*mass) - (-v_0*x2*2*alpha*q+v_0**2*4*alpha**2*q2*x2))#0.5*dt*(2.0*alp*D*(x*(3.0*x2 - 2*x*x) - x2)  - (alp*x/mass)*(p2 - 2.0*p*p) )
    return xp


def QHD_eom(potential_type):
    
    # Again, we must first define all variables as symbols. #

    q, p, p2, x, x2, xp, dt, mass, D, alpha, sigma, q2 = symbols("q, p, p2, x, x2, xp, dt, mass, D, alpha, sigma q2")
    q_0 = 0.0
    alp = alpha
    
    if potential_type == "gaussian":
        potential_sym = -v_0*exp(-alpha*q**2)

        px = xp
        q1 = str(time_deriv(q, 1)*dt + q)

        q21 = str(time_deriv(q, 2)*dt + q2)

        x = exp(-alpha*q**2)
        x1 = sympify(str(time_deriv(x, 1)).replace("exp(-alpha*q**2)", "x"))
        x1 = str((symmetrize(x1))*(0.5)*dt + Symbol("x"))
        x1 = str(x1).replace("p*q*x", "q*xp")

        p1 = str(time_deriv(p, 1)).replace("v(q)", str(potential_sym))
        p1 = expand(sympify(p1).doit())
        p1 = str(p1).replace("exp(-2.0*q**2)", "x2").replace("alpha**2", "0")
        p1 = sympify(p1)
        p1 = str(symmetrize(p1)*(0.5)*dt + p).replace("exp(-alpha*q**2)", "x")

        x = Symbol("x")
        p21 = str(time_deriv(p, 2)).replace("v(q)", str(potential_sym))
        p21 = expand(sympify(p21).doit())
        p21 = str(-p21).replace("exp(-alpha*q**2)", "x").replace("exp(-2*alpha*q**2)", "x2").replace("alpha**2", "0")
        p21 = str(symmetrize(p21)*(0.5)*dt + p2).replace("exp(-alpha*q**2)", "x").replace("exp(-2.0*q**2)", "x2")
        p21 = str(p21).replace("p*q*v_0*x", "v_0*q*xp")
       
       
    if potential_type == "morse":
        potential_sym = D * (1 - exp(-alpha*(q-q_0)))**2
        
        px = xp
        q1 = str(time_deriv(q, 1)*dt + q)

        x = exp(-alpha*q)
        x1 = sympify(str(time_deriv(x, 1)).replace("exp(-alpha*q)", "x"))
        x1 = str((symmetrize(x1))*(0.5)*dt + Symbol("x"))

        p1 = str(time_deriv(p, 1)).replace("v(q)", str(potential_sym))
        p1 = expand(sympify(p1).doit())
        p1 = str(p1).replace("exp(-2*alpha*q)", "x2")#.replace("alpha**2", "0")
        p1 = sympify(p1)
        p1 = str(symmetrize(p1)*(0.5)*dt + p).replace("exp(-alpha*q)", "x")

        x = Symbol("x")
        p21 = str(time_deriv(p, 2)).replace("v(q)", str(potential_sym))
        p21 = expand(sympify(p21).doit())
        p21 = str(-p21).replace("exp(-alpha*q)", "x").replace("exp(-2*alpha*q)", "x2").replace("alpha**2", "0")
        p21 = str(symmetrize(p21)*(0.5)*dt + p2).replace("exp(-alpha*q)", "x").replace("exp(-2*alpha*q)", "x2")


    if potential_type == "double_well":
        potential_sym = m*(q**2 - B)**2
        
        q1 = str(QHD_int(q, 1, dt))
        p1 = str(sympify(str(QHD_int(p, 1, dt)).replace("v(q)", str(potential_sym))))
        
        p = eval(p1)
        q = eval(q1)
        p = eval(p1)

In [46]:
QHD_eom("morse")

In [47]:
potential_type = "morse"

hbar = 0.6582  # eV * fs
convert = (1.0/17.586)
mass, q0, p0, s0, ps0, T, dt = 2980.0*convert, 0.15, 0.0, 0.05, 0.0, 500, 0.1
alp = 2.567
alpha = alp


D = 10

q, p = q0, p0
q_0 = 0.0
q2 = q*q + s0*s0
pq = ps0*s0 + q*p
p2 = p*p + ps0*ps0 + (0.5*hbar/s0)**2
x = math.exp(-alp*q0)*math.exp(0.5*alp*alp*s0*s0)
x2 = math.exp(-2.0*alp*q0)*math.exp(2.0*alp*alp*s0*s0)
xq = x*(q-alp*s0*s0)
xp = x*(p - s0*ps0*alp)

if potential_type == "gaussian":
    xp = gaus_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
    x2 = gaus_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
    x = eval(x1)
    p2 = eval(p21)
    q2 = eval(q21)
    p = eval(p1)
    q = eval(q1)
    p = eval(p1)
    q2 = eval(q21)
    p2 = eval(p21)
    x = eval(x1)
    x2 = gaus_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
    xp = gaus_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)

if potential_type == "morse":
    xp = morse_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
    x2 = morse_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
    x = eval(x1)
    p2 = eval(p21)
    p = eval(p1)
    q = eval(q1)
    p = eval(p1)
    p2 = eval(p21)
    x = eval(x1)
    x2 = morse_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
    xp = morse_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)

if potential_type == "double_well":
    p = eval(str(p1))
    q = eval(str(q1))
    p = eval(str(p1))

        
        
### Time how long it takes to compute ###

import time 
begin = time.time()


### Compute the x values ###

dt = 0.1 # step size
t_i = 0.0 # start
t_f = 1000 # finish


### Compute the y values ###

i=0
while i<1:        
    s = (q2 - q*q)
    if s>0.0:
        s = math.sqrt(s)
    else: 
        s = 0.0
    if s>0.0:
        ps = (pq - p*q)/s
    else: 
        ps = 0.0
    i = i+1

q_list = []
p_list = []
q2_list = []
p2_list = []

if potential_type == "gaussian":
    while t_i<=t_f:
        xp = gaus_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
        x2 = gaus_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
        x = eval(x1)
        p2 = eval(p21)
        q2 = eval(q21)
        p = eval(p1)
        q = eval(q1)
        p = eval(p1)
        q2 = eval(q21)
        p2 = eval(p21)
        x = eval(x1)
        x2 = gaus_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
        xp = gaus_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)



if potential_type == "morse":
    while t_i <= t_f:
        xp = morse_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
        x2 = morse_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
        x = eval(x1)
        p2 = eval(p21)
        p = eval(p1)
        q = eval(q1)
        p = eval(p1)
        p2 = eval(p21)
        x = eval(x1)
        x2 = morse_integrate_x2(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)
        xp = morse_integrate_xp(q,p,pq,q2,p2,x,x2,xp,xq,0.5*dt,mass,alp,D)

if potential_sym == "double_well":
     while t_i <= t_f:
            p = eval(str(p1))
            q = eval(str(q1))
            p = eval(str(p1))

t_i = t_i + dt
i=0
while i<1:
    s = (q2 - q*q)
    if s>0.0:
        s = math.sqrt(s)
    else: 
        s = 0.0
    if s>0.0:
        ps = (pq - p*q)/s
    else: 
        ps = 0.0

    i = i+1
    q_list.append(q)
    p_list.append(p)
    q2_list.append(q2)
    p2_list.append(p2)

### Compute the end time ###

time.sleep(1)
# store end time
end = time.time()

# total time taken
print(f"Total runtime is {end - begin}")

NameError: name 'x1' is not defined